In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, glob, os, scipy.stats as stats
from neo.rawio import BlackrockRawIO
from neo.io import BlackrockIO


In [28]:
from neo.io import BlackrockIO
import scipy.io as sio

io = BlackrockIO("../results/202512/datafile_202512b002_DI")
block = io.read_block(lazy=False)

# Example: get first segment, all analog signals
seg = block.segments[0]
signals = [asig.as_array() for asig in seg.analogsignals]

sio.savemat("pt3_nsx_from_python.mat", {"signals": signals})


OSError: NEV file type MATLAB 5 is not supported

In [4]:
from scipy.io import loadmat
mat = loadmat("../results/202512/datafile_202512b002_DI.mat")  # actually a .mat
print(mat.keys())


dict_keys(['__header__', '__version__', '__globals__', 'NEV', 'nevFile', 'nevList'])


In [10]:
with h5py.File("../results/202512/datafile_202512b002_DI.ns6", "r") as f:
    def walk(name, obj):
        print(name, "->", type(obj))
    f['NSX'].visititems(walk)


Data -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo -> <class 'h5py._hl.group.Group'>
ElectrodesInfo/AnalogUnits -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/ConnectorBank -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/ConnectorPin -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/ElectrodeID -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/HighFilterType -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/HighFreqCorner -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/HighFreqOrder -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/Label -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/LowFilterType -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/LowFreqCorner -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/LowFreqOrder -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/MaxAnalogValue -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/MaxDigiValue -> <class 'h5py._hl.dataset.Dataset'>
ElectrodesInfo/MinAnalogValue -> <class 'h5py._hl

In [22]:
import h5py, numpy as np

def _decode_mat_string(obj):
    a = np.array(obj[()])
    if a.dtype == np.uint16:
        return a.tobytes().decode("utf-16le", errors="ignore").strip("\x00 ").strip()
    return a.tobytes().decode("utf-8", errors="ignore").strip("\x00 ").strip()

def _from_refs_numeric(f, ds):
    """Return a 1D float array from a dataset that may contain h5 references or plain nums."""
    val = ds[()]
    A = np.array(val)
    if A.dtype.kind != 'O':  # plain numeric
        return np.array(A, dtype=np.float64).squeeze()
    # object array of references
    out = []
    for ref in A.flatten():
        if isinstance(ref, h5py.Reference) and ref:
            x = np.array(f[ref][()]).astype(np.float64).squeeze()
            out.append(x.item() if x.size == 1 else np.nan)
        else:
            out.append(np.nan)
    return np.array(out, dtype=np.float64)

def _from_refs_strings(f, ds):
    """Return list[str] from dataset that may be references or fixed-width bytes."""
    val = ds[()]
    A = np.array(val)
    # reference array
    if A.dtype.kind == 'O':
        out = []
        for ref in A.flatten():
            if isinstance(ref, h5py.Reference) and ref:
                out.append(_decode_mat_string(f[ref]))
            else:
                out.append("")
        return out
    # fixed-width bytes fallback
    labels = []
    for lbl in A:
        b = np.asarray(lbl).ravel().view(np.uint8)
        b = b[b != 0]
        labels.append(b.tobytes().decode("utf-8", errors="ignore").strip())
    return labels

def load_nsx_mat(path, return_scaling=False):
    with h5py.File(path, "r") as f:
        nsx = f["NSX"]

        data   = nsx["Data"][()]                                   # (C, T)
        sr     = nsx["MetaTags"]["SamplingFreq"][()].item()
        start  = nsx["MetaTags"]["Timestamp"][()].item()
        n_samp = nsx["MetaTags"]["DataPoints"][()].item()
        ts = (np.arange(n_samp, dtype=np.float64) + start) / float(sr)

        # Labels (refs or bytes)
        labels = _from_refs_strings(f, nsx["ElectrodesInfo"]["Label"])

        scales_uV = None
        if return_scaling:
            ei = nsx["ElectrodesInfo"]
            # These may be reference arrays:
            maxA = _from_refs_numeric(f, ei["MaxAnalogValue"])
            minA = _from_refs_numeric(f, ei["MinAnalogValue"])
            maxD = _from_refs_numeric(f, ei["MaxDigiValue"])
            minD = _from_refs_numeric(f, ei["MinDigiValue"])
            units = _from_refs_strings(f, ei["AnalogUnits"])

            analog_per_count = (maxA - minA) / (maxD - minD)
            scales_uV = np.empty_like(analog_per_count, dtype=np.float64)
            for i, u in enumerate(units):
                ul = (u or "").lower()
                if ul.startswith("uv"):
                    scales_uV[i] = analog_per_count[i]
                elif ul.startswith("mv"):
                    scales_uV[i] = analog_per_count[i] * 1e3
                elif ul.startswith("v"):
                    scales_uV[i] = analog_per_count[i] * 1e6
                else:
                    scales_uV[i] = np.nan

    return (data, ts, sr, labels, scales_uV) if return_scaling else (data, ts, sr, labels)


In [23]:
data, ts, sr, labels, scales_uV = load_nsx_mat("../results/202512/datafile_202512b002_DI.ns6", return_scaling=True)
print(data.shape, sr)
print(labels[:8])
print(scales_uV[:8])
# Example: convert channel 0 to µV (if scale known)
ch0_uV = data[0].astype(np.float64) * scales_uV[0] if np.isfinite(scales_uV[0]) else data[0]


(48890893, 32) 30000.0
['chan97', 'chan98', 'chan99', 'chan100', 'chan101', 'chan102', 'chan103', 'chan104']
[0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]


In [5]:
# load data
io_pt2    = BlackrockIO(filename='../results/202512/datafile_202512b002_DI')
block_pt2 = io_pt2.read_block(lazy=False)
seg_pt2   = block_pt2.segments[0]


OSError: Unsupported NSX file type.

In [7]:
subj_spikes = seg_pt2.spiketrains
len(subj_spikes), subj_spikes[0]

(32,
 SpikeTrain containing 550 spikes; units s; datatype float64 
 name: 'ch97#0'
 description: 'SpikeTrain channel_id: 97, unit_id: 0'
 annotations: {'id': np.str_('Unit 97000'),
   'channel_id': 97,
   'unit_id': 0,
   'unit_tag': 'unclassified'}
 time: 0.0034 s to 1567.5468666666668 s)

In [4]:
trials_df = pd.read_csv('../results/asymmetry_final_subj-202512_difficulty-patients_order-C_2025-07-26_12h47.29.109.csv')
trials_df

,thisN,thisTrialN,thisRepN,blockN,run,condition,trial_key,stim_file_pos,stim_pos,noise_pos,...,blocks.block_end_resp.duration,subj,difficulty,sess_type,date,expName,psychopyVersion,frameRate,expStart,Unnamed: 112
0,0.0,0.0,0.0,1.0,1.0,baseline,21.0,2.1,0.02,3.0,...,NaN,202512.0,patients,C,2025-07-26_12h47.29.109,asymmetry_final,2024.2.4,60.0,2025-07-26 12h47.44.103153 -0600,NaN
1,1.0,1.0,0.0,1.0,1.0,baseline,35.0,3.5,0.30,3.0,...,NaN,202512.0,patients,C,2025-07-26_12h47.29.109,asymmetry_final,2024.2.4,60.0,2025-07-26 12h47.44.103153 -0600,NaN
2,2.0,2.0,0.0,1.0,1.0,baseline,26.0,2.7,0.14,3.0,...,NaN,202512.0,patients,C,2025-07-26_12h47.29.109,asymmetry_final,2024.2.4,60.0,2025-07-26 12h47.44.103153 -0600,NaN
3,3.0,3.0,0.0,1.0,1.0,baseline,5.0,0.5,-0.30,1.0,...,NaN,202512.0,patients,C,2025-07-26_12h47.29.109,asymmetry_final,2024.2.4,60.0,2025-07-26 12h47.44.103153 -0600,NaN
4,4.0,4.0,0.0,1.0,1.0,baseline,37.0,3.7,0.34,3.0,...,NaN,202512.0,patients,C,2025-07-26_12h47.29.109,asymmetry_final,2024.2.4,60.0,2025-07-26 12h47.44.103153 -0600,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,36.0,36.0,0.0,6.0,2.0,flat_comp,201.0,1.1,-0.18,3.0,...,NaN,202512.0,patients,C,2025-07-26_12h47.29.109,asymmetry_final,2024.2.4,60.0,2025-07-26 12h47.44.103153 -0600,NaN
237,37.0,37.0,0.0,6.0,2.0,flat_comp,237.0,3.9,0.38,1.0,...,NaN,202512.0,patients,C,2025-07-26_12h47.29.109,asymmetry_final,2024.2.4,60.0,2025-07-26 12h47.44.103153 -0600,NaN
238,38.0,38.0,0.0,6.0,2.0,flat_comp,217.0,2.7,0.14,3.0,...,NaN,202512.0,patients,C,2025-07-26_12h47.29.109,asymmetry_final,2024.2.4,60.0,2025-07-26 12h47.44.103153 -0600,NaN
239,39.0,39.0,0.0,6.0,2.0,flat_comp,204.0,1.5,-0.10,3.0,...,NaN,202512.0,patients,C,2025-07-26_12h47.29.109,asymmetry_final,2024.2.4,60.0,2025-07-26 12h47.44.103153 -0600,NaN
